## Problem Analysis

## Data Collection

### Importing libraries

In [59]:
# Import all necessary libraries
import pandas as pd
import ydata_profiling
import os
import numpy as np
import gc
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import roc_auc_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

# Configurar pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

### Collecting data

In [23]:
# Paso 1: Ingesta y unificación de datos con polars

# Rutas de los archivos
setA_path = r"C:\repos\physionet-sepsis-forecasting\data\raw\all_patients_setA.parquet"
setB_path = r"C:\repos\physionet-sepsis-forecasting\data\raw\all_patients_setB.parquet"
unified_path = r"C:\repos\physionet-sepsis-forecasting\data\raw\all_patients_unified.parquet"

In [33]:
# Leer ambos datasets con pandas
df_a = pd.read_parquet(setA_path)
df_b = pd.read_parquet(setB_path)

In [34]:
# Mostrar las columnas que NO tienen en común
print("Columnas en Set A pero no en Set B:", set(df_a.columns) - set(df_b.columns))
print("Columnas en Set B pero no en Set A:", set(df_b.columns) - set(df_a.columns))

Columnas en Set A pero no en Set B: {'subHR'}
Columnas en Set B pero no en Set A: set()


In [35]:
# Asegurar que ambos tengan las mismas columnas eliminando las que no coinciden
common_cols = list(set(df_a.columns) & set(df_b.columns))
df_a = df_a[common_cols]
df_b = df_b[common_cols]

In [36]:
# Unificar
df = pd.concat([df_a, df_b])

# Guardar el dataset unificado
df.to_parquet(unified_path)
print(f"Dataset unificado guardado en {unified_path}")

Dataset unificado guardado en C:\repos\physionet-sepsis-forecasting\data\raw\all_patients_unified.parquet


In [37]:
# Limpiar memoria eliminando df_a y df_b
del df_a
del df_b
gc.collect()

1714

## EDA

In [52]:
# Paso 2: Análisis exploratorio
df = pd.read_parquet(unified_path)
df.shape

(1552210, 42)

In [39]:
# Ver las primeras filas
df.head()

,Chloride,Unit1,Lactate,Bilirubin_total,Glucose,Creatinine,SBP,Temp,WBC,HospAdmTime,TroponinI,HR,PTT,SepsisLabel,Potassium,Age,HCO3,Alkalinephos,BaseExcess,Hgb,FiO2,Calcium,Phosphate,Resp,EtCO2,MAP,Gender,Platelets,Fibrinogen,patient_id,Magnesium,SaO2,Hct,BUN,ICULOS,DBP,pH,Unit2,Bilirubin_direct,PaCO2,AST,O2Sat
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,NaN,NaN,NaN,0.0,NaN,83.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,p000001,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,98.0,NaN,NaN,-0.03,NaN,97.0,NaN,0.0,NaN,83.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,NaN,75.33,0.0,NaN,NaN,p000001,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,95.0
2,NaN,NaN,NaN,NaN,NaN,NaN,122.0,NaN,NaN,-0.03,NaN,89.0,NaN,0.0,NaN,83.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,86.00,0.0,NaN,NaN,p000001,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,99.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,NaN,90.0,NaN,0.0,NaN,83.14,NaN,NaN,24.0,NaN,NaN,NaN,NaN,30.0,NaN,NaN,0.0,NaN,NaN,p000001,NaN,NaN,NaN,NaN,4.0,NaN,7.36,NaN,NaN,100.0,NaN,95.0
4,NaN,NaN,NaN,NaN,NaN,NaN,122.0,NaN,NaN,-0.03,NaN,103.0,NaN,0.0,NaN,83.14,NaN,NaN,NaN,NaN,0.28,NaN,NaN,24.5,NaN,91.33,0.0,NaN,NaN,p000001,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,88.5


In [40]:
# Descripción estadística rápida
df.describe()

,Chloride,Unit1,Lactate,Bilirubin_total,Glucose,Creatinine,SBP,Temp,WBC,HospAdmTime,TroponinI,HR,PTT,SepsisLabel,Potassium,Age,HCO3,Alkalinephos,BaseExcess,Hgb,FiO2,Calcium,Phosphate,Resp,EtCO2,MAP,Gender,Platelets,Fibrinogen,Magnesium,SaO2,Hct,BUN,ICULOS,DBP,pH,Unit2,Bilirubin_direct,PaCO2,AST,O2Sat
count,70466.000000,940250.000000,41446.000000,23141.000000,265516.000000,94616.000000,1.325945e+06,525226.000000,99447.000000,1.552202e+06,14781.000000,1.398786e+06,45699.000000,1.552210e+06,144525.000000,1.552210e+06,65028.000000,24941.000000,84145.000000,114591.000000,129365.000000,91331.000000,62301.000000,1.313875e+06,57636.000000,1.358940e+06,1.552210e+06,92209.000000,10242.000000,97951.000000,53561.000000,137433.000000,106568.000000,1.552210e+06,1.065656e+06,107573.000000,940250.000000,2990.000000,86301.000000,25183.000000,1.349474e+06
mean,105.827910,0.496571,2.646666,2.114059,136.932283,1.510699,1.237505e+02,36.977228,11.446405,-5.612512e+01,8.290099,8.458154e+01,41.231193,1.798468e-02,4.135528,6.200947e+01,24.075481,102.483661,-0.689919,10.430833,0.554839,7.557531,3.544238,1.872650e+01,32.957657,8.240010e+01,5.592690e-01,196.013911,287.385706,2.051450,92.654188,30.794093,23.915452,2.699499e+01,6.383056e+01,7.378934,0.503429,1.836177,41.021869,260.223385,9.719395e+01
std,5.880462,0.499989,2.526214,4.311468,51.310728,1.805603,2.323156e+01,0.770014,7.731013,1.622569e+02,24.806235,1.732537e+01,26.217669,1.328956e-01,0.642150,1.638622e+01,4.376504,120.122746,4.294297,1.968661,11.123207,2.433152,1.423286,5.098194e+00,7.951662,1.634175e+01,4.964749e-01,103.635366,153.002908,0.397898,10.892986,5.491749,19.994317,2.900542e+01,1.395601e+01,0.074568,0.499989,3.694082,9.267242,855.746795,2.936924e+00
min,26.000000,0.000000,0.200000,0.100000,10.000000,0.100000,2.000000e+01,20.900000,0.100000,-5.366860e+03,0.010000,2.000000e+01,12.500000,0.000000e+00,1.000000,1.400000e+01,0.000000,7.000000,-32.000000,2.200000,-50.000000,1.000000,0.200000,1.000000e+00,10.000000,2.000000e+01,0.000000e+00,1.000000,34.000000,0.200000,23.000000,5.500000,1.000000,1.000000e+00,2.000000e+01,6.620000,0.000000,0.010000,10.000000,3.000000,2.000000e+01
25%,102.000000,0.000000,1.260000,0.500000,106.000000,0.700000,1.070000e+02,36.500000,7.600000,-4.705000e+01,0.040000,7.200000e+01,27.800000,0.000000e+00,3.700000,5.168000e+01,22.000000,54.000000,-3.000000,9.100000,0.400000,7.700000,2.600000,1.500000e+01,28.000000,7.100000e+01,0.000000e+00,126.000000,184.000000,1.800000,94.000000,27.000000,12.000000,1.100000e+01,5.400000e+01,7.340000,0.000000,0.200000,35.000000,22.000000,9.600000e+01
50%,106.000000,0.000000,1.800000,0.900000,127.000000,0.940000,1.210000e+02,37.000000,10.300000,-6.030000e+00,0.300000,8.350000e+01,32.400000,0.000000e+00,4.100000,6.400000e+01,24.000000,74.000000,0.000000,10.300000,0.500000,8.300000,3.300000,1.800000e+01,33.000000,8.000000e+01,1.000000e+00,181.000000,250.000000,2.000000,97.000000,30.300000,17.000000,2.100000e+01,6.200000e+01,7.380000,1.000000,0.445000,40.000000,41.000000,9.800000e+01
75%,109.000000,1.000000,3.000000,1.700000,153.000000,1.430000,1.380000e+02,37.500000,13.800000,-4.000000e-02,3.980000,9.550000e+01,42.800000,0.000000e+00,4.400000,7.400000e+01,26.800000,108.000000,1.000000,11.700000,0.600000,8.700000,4.100000,2.150000e+01,38.000000,9.200000e+01,1.000000e+00,244.000000,349.000000,2.200000,98.000000,34.100000,28.000000,3.400000e+01,7.200000e+01,7.430000,1.000000,1.700000,45.000000,111.000000,9.950000e+01
max,145.000000,1.000000,31.000000,49.600000,988.000000,46.600000,3.000000e+02,50.000000,440.000000,2.399000e+01,440.000000,2.800000e+02,250.000000,1.000000e+00,27.500000,1.000000e+02,55.000000,3833.000000,100.000000,32.000000,4000.000000,27.900000,18.800000,1.000000e+02,100.000000,3.000000e+02,1.000000e+00,2322.000000,1760.000000,9.800000,100.000000,71.700000,268.000000,3.360000e+02,3.000000e+02,7.930000,1.000000,37.500000,100.000000,9961.000000,1.000000e+02


In [44]:
# Conteo de valores nulos por columna con porcentaje
null_counts = df.isnull().sum()
null_percent = (null_counts / len(df)) * 100
null_df = pd.DataFrame({'null_count': null_counts, 'null_percent': null_percent})
null_df = null_df[null_df['null_count'] >= 0].sort_values(by='null_percent', ascending=False)
print(null_df)

                  null_count  null_percent
Bilirubin_direct     1549220     99.807371
Fibrinogen           1541968     99.340167
TroponinI            1537429     99.047745
Bilirubin_total      1529069     98.509158
Alkalinephos         1527269     98.393194
AST                  1527027     98.377604
Lactate              1510764     97.329872
PTT                  1506511     97.055875
SaO2                 1498649     96.549372
EtCO2                1494574     96.286843
Phosphate            1489909     95.986303
HCO3                 1487182     95.810618
Chloride             1481744     95.460279
BaseExcess           1468065     94.579020
PaCO2                1465909     94.440121
Calcium              1460879     94.116067
Platelets            1460001     94.059502
Creatinine           1457594     93.904433
Magnesium            1454259     93.689578
WBC                  1452763     93.593199
BUN                  1445642     93.134434
pH                   1444637     93.069688
Hgb        

In [47]:
# Analizar desbalance de clases en la variable objetivo SepsisLabel
sepsis_counts = df['SepsisLabel'].value_counts()
sepsis_percent = (sepsis_counts / len(df)) * 100
sepsis_classes = pd.DataFrame({'sepsis_counts': sepsis_counts, 'sepsis_percent': sepsis_percent})
sepsis_classes = sepsis_classes[sepsis_classes['sepsis_counts'] >= 0].sort_values(by='sepsis_percent', ascending=False)

print(sepsis_classes)

             sepsis_counts  sepsis_percent
SepsisLabel                               
0.0                1524294       98.201532
1.0                  27916        1.798468


In [48]:
# Generar un reporte con ydata-profiling

profile = ydata_profiling.ProfileReport(df, title="Reporte de Análisis Exploratorio", explorative=True, minimal=True)
# Si el directorio no existe, crearlo
os.makedirs(os.path.dirname(r"C:\repos\physionet-sepsis-forecasting\data\reports"), exist_ok=True)
# Guardar el reporte
profile_path = r"C:\repos\physionet-sepsis-forecasting\data\reports\eda_report.html"
profile.to_file(profile_path)

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 26.31it/s]


## Data Cleaning & Preprocessing

### Imputation

In [53]:
# Imputar valores nulos con la media usando pandas y para las variables categóricas con la moda
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for col in numeric_cols:
    mean_value = df[col].mean()
    df[col] = df[col].fillna(mean_value)

categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
for col in categorical_cols:
    mode_value = df[col].mode()[0]
    df[col] = df[col].fillna(mode_value)

### Data Scalation

In [54]:
# Aplicar StandardScaler de sklearn a las columnas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# Excluir columnas que no deben ser escaladas
exclude_cols = ['SepsisLabel', 'patient_id', 'ICULOS']
numeric_cols = [col for col in numeric_cols if col not in exclude_cols]
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

### PCA

In [55]:
categorical_cols

['patient_id']

In [60]:
# Crear PCA para reducción de dimensionalidad creando df sin la columna objetivo, la columna patient_id y la columna de tiempo ICULOS
pca = PCA(n_components=0.95)  # Mantener el 95% de la varianza
features = df.drop(columns=['SepsisLabel', 'patient_id', 'ICULOS']) 
# Ajustar según el nombre real de la columna ID
features_pca = pca.fit_transform(features)
print(f"Original number of features: {features.shape[1]}")
print(f"Reduced number of features after PCA: {features_pca.shape[1]}")

Original number of features: 39
Reduced number of features after PCA: 32


In [63]:
# Imprimir las nuevas features PCA
features_pca_df = pd.DataFrame(features_pca, columns=[f'PC{i+1}' for i in range(features_pca.shape[1])])
print(features_pca_df.shape)
features_pca_df.head()

(1552210, 32)


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32
0,-0.123064,-0.029543,0.142505,0.120093,-0.052242,-0.087159,-0.155298,-0.272492,-0.309747,-0.050145,0.931490,-0.104184,-0.007021,-0.539897,0.713123,-0.501857,0.036661,-0.224363,-0.265463,-0.542381,0.273190,-0.021102,0.056892,-0.084623,-0.137621,-0.016891,-0.026107,0.005941,-0.083408,0.429037,0.040965,-0.009478
1,-0.789446,-0.228222,0.675249,0.281556,-0.099754,-0.029427,0.300584,0.515659,-0.185963,-0.256120,0.836313,-0.146537,-0.041683,-0.463266,0.600306,-0.506419,0.045059,-0.150722,-0.292934,-0.430488,0.277385,0.019641,-0.580837,0.004692,-0.131852,0.188845,-0.068946,-0.118287,-0.073026,0.929342,0.053358,-0.016869
2,0.014555,0.025953,0.041635,0.096440,-0.109083,-0.077429,-0.058951,0.036207,-0.283142,-0.029096,0.867648,-0.087519,-0.026721,-0.792702,0.689297,-0.468095,0.022012,-0.205364,-0.326825,-0.698144,0.332258,-0.027933,0.428618,-0.035524,-0.165604,-0.659961,0.389910,0.053687,-0.175552,0.273831,0.041368,0.015204
3,2.579250,-3.492402,-1.754014,10.292895,9.374868,-3.562780,16.456295,-8.193478,14.697793,0.537361,2.060404,-17.457827,2.158041,-3.816088,-2.791320,-3.857569,0.555117,-3.411087,2.320667,-1.128697,3.597547,1.404556,-1.165497,2.073437,2.259078,-3.488441,-2.790034,10.718579,-2.985655,-0.846346,1.175720,0.062475
4,0.286442,0.199674,0.226638,-0.060309,-0.111525,-0.062679,1.163566,1.571447,-0.418847,-1.115356,2.263066,-0.521561,-0.251348,0.032512,0.737198,-0.694443,-0.006784,-0.337366,-0.264212,-0.115577,0.057176,-0.121472,-1.609570,0.128556,-0.007647,0.856901,-0.252471,-0.238448,0.005595,0.565791,0.017684,0.014394
